# 使用FP-growth高效发现频繁项集

FP（Frequent Pattern）频繁模式，一种存储项集出现次数的数据结构

growth —— 生长，一种可以让树自增高的算法
* Apriori算法产生候选项集，扫描数据集来检查他们是否频繁
* FP-growth算法改进Apriori，将数据集存储在FP树中，通过查找元素项的条件基以及构建条件FP树来发现频繁项集。

## 构建FP树的数据结构
* FP树的结点定义

In [1]:
class tree_node:
    def __init__(self, name, num, parent):
        self.name = name
        self.num = num
        self.node_link = None
        self.parent = parent
        self.child = {}

* 节点数增加

In [2]:
def inc(self, num):
    self.num += num

* 遍历节点

In [3]:
def disp(self, ind=1):
    print(' ' * ind,self.name, ' ', self.num)
    for child in self.child.values():
        disp(child, ind + 1)

In [4]:
root = tree_node('python', 8, None)
child1 = tree_node('java', 1, root)
root.child['java'] = child1
child2 = tree_node('c++', 5, root)
root.child['c++'] = child2

In [5]:
disp(root)

  python   8
   java   1
   c++   5


## 构建PF树

* 对数据集进行两次遍历

第一次遍历：

    计算每个元素项的频率
    去掉不满足最小支持度的项，排序
第二次遍历：

    自上而下遍历树节点
    如果当前元素节点存在，则增加此节点的值
    如果当前元素节点不存在，则添加一个分支节点

生长函数，元素项存在则计数，不存在则生长

In [6]:
def update_tree(item, intree, header_table, count):
    if item[0] in intree.child:
        inc(intree.child[item[0]], count)
    else:
        intree.child[item[0]] = tree_node(item[0], count, intree)
        if header_table[item[0]][1] == None:
            header_table[item[0]][1] = intree.child[item[0]]
        else:
            update_header(header_table[item[0]][1], intree.child[item[0]])
    if len(item) > 1:
        update_tree(item[1::], intree.child[item[0]], header_table, count)

In [7]:
def update_header(node_to_test, target_node):
    while node_to_test.node_link != None:
        node_to_test = node_to_test.node_link
    node_to_test.node_link = target_node

In [26]:
def create_tree(dataset, min_sup=1):
    header_table = {}
    for trans in dataset:
        for item in trans:
            header_table[item] = header_table.get(item, 0) + dataset[trans]
    header_table_copy = header_table.copy()
    for k in header_table_copy.keys():
        if header_table[k] < min_sup:
            del(header_table[k])
    freq_item_set = set(header_table.keys())
    if len(freq_item_set) == 0:
        return None, None
    for k in header_table:
        header_table[k] = [header_table[k], None]
    ret_tree = tree_node('Null Set', 1, None)
    for tran_set, count in dataset.items():
        localD = {}
        for item in tran_set:
            if item in freq_item_set:
                localD[item] = header_table[item][0]
        if len(localD) > 0:
            ordered_item = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]            
            update_tree(ordered_item, ret_tree, header_table, count)
#     print(localD)
#     print(header_table)
    return ret_tree, header_table

## 载入数据

In [9]:
def load_simpdat():
    simpdat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpdat

def create_initset(dataset):
    retdict = {}
    for trans in dataset:
        retdict[frozenset(trans)] = 1
    return retdict

In [10]:
simpdat = load_simpdat()
initset = create_initset(simpdat)
initset

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [11]:
mytree, myheadtab = create_tree(initset, 3)

{'y': 3, 't': 3, 's': 3, 'z': 5, 'x': 4}
{'z': [5, <__main__.tree_node object at 0x0000022C52BCEF88>], 'r': [3, <__main__.tree_node object at 0x0000022C52BCE908>], 'y': [3, <__main__.tree_node object at 0x0000022C52BCE688>], 't': [3, <__main__.tree_node object at 0x0000022C52BCE608>], 's': [3, <__main__.tree_node object at 0x0000022C52BCE448>], 'x': [4, <__main__.tree_node object at 0x0000022C52BCEC08>]}


In [12]:
disp(mytree)

  Null Set   1
   z   5
    r   1
    x   3
     y   3
      t   3
       s   2
       r   1
   x   1
    s   1
     r   1


## 从FP树中挖掘频繁项集
基本步骤：
    
    1.从FP树中获取条件模式基
    2.利用条件模式基构建PF树
    3.重复1，2直到树包含一个元素项为止

* 条件模式基（conditional pattern base）是以所查找元素项为结尾的路径集合
* 前缀路径（prefix path）介于所查找元素项与根节点之间的所有内容


迭代上溯整棵树

In [13]:
def ascend_tree(leaf_node, prefix_path):
    if leaf_node.parent != None:
        prefix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, prefix_path)

In [14]:
def find_prefixpath(basepath, tree_node):
    cond_path = {}
    while tree_node != None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            cond_path[frozenset(prefix_path[1:])] = tree_node.num
        tree_node = tree_node.node_link
    return cond_path

In [15]:
find_prefixpath('x', myheadtab['x'][1])

{frozenset({'z'}): 3}

In [16]:
find_prefixpath('z', myheadtab['z'][1])

{}

In [17]:
find_prefixpath('r', myheadtab['r'][1])

{frozenset({'z'}): 1,
 frozenset({'s', 'x'}): 1,
 frozenset({'t', 'x', 'y', 'z'}): 1}

## 创建条件FP树

In [35]:
def mine_tree(intree, headertable, minsup, prefix, freq_itemlist):
    bigL = [v[0] for v in sorted(headertable.items(), key=lambda p:p[0])]
    print(bigL)
    for basepath in bigL:
        newfreqset = prefix.copy()
        newfreqset.add(basepath)
        freq_itemlist.append(newfreqset)
        cond_pattbase = find_prefixpath(basepath, headertable[basepath][1])
        mycondtree, myhead = create_tree(cond_pattbase, minsup)
        if myhead != None:
            print('conditional tree for: ',newfreqset)
            disp(mycondtree, 1) 
            mine_tree(mycondtree, myhead, minsup, newfreqset, freq_itemlist)

In [36]:
freq_items = []
mine_tree(mytree, myheadtab, 3, set([]), freq_items)

['r', 's', 't', 'x', 'y', 'z']
conditional tree for:  {'s'}
  Null Set   1
   x   3
['x']
conditional tree for:  {'t'}
  Null Set   1
   x   3
    y   3
     z   3
['x', 'y', 'z']
conditional tree for:  {'t', 'y'}
  Null Set   1
   x   3
['x']
conditional tree for:  {'t', 'z'}
  Null Set   1
   x   3
    y   3
['x', 'y']
conditional tree for:  {'t', 'y', 'z'}
  Null Set   1
   x   3
['x']
conditional tree for:  {'x'}
  Null Set   1
   z   3
['z']
conditional tree for:  {'y'}
  Null Set   1
   x   3
    z   3
['x', 'z']
conditional tree for:  {'y', 'z'}
  Null Set   1
   x   3
['x']


## 示例：在Twitter源中发现一些共现词（API不可用）

## 示例：从新闻网站点击流中挖掘

In [39]:
parsedat = [line.split() for line in open('kosarak.dat').readlines()]
parsedat

[['1', '2', '3'],
 ['1'],
 ['4', '5', '6', '7'],
 ['1', '8'],
 ['9', '10'],
 ['11', '6', '12', '13', '14', '15', '16'],
 ['1', '3', '7'],
 ['17', '18'],
 ['11', '6', '19', '20', '21', '22', '23', '24'],
 ['1', '25', '3'],
 ['26', '3'],
 ['11',
  '27',
  '6',
  '3',
  '28',
  '7',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37'],
 ['6', '2', '38'],
 ['39',
  '11',
  '27',
  '1',
  '40',
  '6',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '3',
  '48',
  '7',
  '49',
  '50',
  '51'],
 ['52', '6', '3', '53'],
 ['54', '1', '6', '55'],
 ['11', '6', '56', '57', '58', '59', '60', '61', '62', '63', '64'],
 ['3'],
 ['1', '65', '66', '67', '68', '3'],
 ['69', '11', '1', '6'],
 ['11', '70', '6'],
 ['6', '3', '71'],
 ['72', '6', '73'],
 ['74'],
 ['75', '76'],
 ['6', '3', '77'],
 ['78', '79', '80', '81'],
 ['82', '6', '83', '7', '84', '85', '86', '87', '88'],
 ['11',
  '27',
  '1',
  '6',
  '89',
  '90',
  '91',
  '92',
  '93',
  '14',
  '94',
  '95',
  '96',
  '9

In [40]:
initset = create_initset(parsedat)
initset

{frozenset({'1', '2', '3'}): 1,
 frozenset({'1'}): 1,
 frozenset({'4', '5', '6', '7'}): 1,
 frozenset({'1', '8'}): 1,
 frozenset({'10', '9'}): 1,
 frozenset({'11', '12', '13', '14', '15', '16', '6'}): 1,
 frozenset({'1', '3', '7'}): 1,
 frozenset({'17', '18'}): 1,
 frozenset({'11', '19', '20', '21', '22', '23', '24', '6'}): 1,
 frozenset({'1', '25', '3'}): 1,
 frozenset({'26', '3'}): 1,
 frozenset({'11',
            '27',
            '28',
            '29',
            '3',
            '30',
            '31',
            '32',
            '33',
            '34',
            '35',
            '36',
            '37',
            '6',
            '7'}): 1,
 frozenset({'2', '38', '6'}): 1,
 frozenset({'1',
            '11',
            '27',
            '3',
            '39',
            '40',
            '41',
            '42',
            '43',
            '44',
            '45',
            '46',
            '47',
            '48',
            '49',
            '50',
            '51',
 

In [41]:
mytree1, myheader1 = create_tree(initset, 100000)

In [42]:
myfreqlist = []
mine_tree(mytree1, myheader1, 100000, set([]), myfreqlist)

['1', '11', '3', '6']
conditional tree for:  {'1'}
  Null Set   1
   6   107404
['6']
conditional tree for:  {'11'}
  Null Set   1
   6   261773
['6']
conditional tree for:  {'3'}
  Null Set   1
   6   186289
    11   117401
   11   9718
['11', '6']
conditional tree for:  {'3', '11'}
  Null Set   1
   6   117401
['6']


In [43]:
myfreqlist

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'6'}]